# Train Deep Speech neural net on text data

1) Global set up - setting up logging and root directory for including the modules, files etc.
2) Text distortion function to generate training data.
3) Load Simple Wikipedia, clean up the data, transform into idx-s and one-hot vectors.
4) Initialize training and validation datasets.
5) Run training.

In [1]:
try:
    with open("global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [2]:
import random
import tensorflow as tf
from keras.utils import to_categorical
import numpy as np
from nltk import tokenize # Text to sentences
import pandas as pd # Train and Validation data generation
import re
import pprint
from src.wikipedia import Wikipedia
#random.seed(12345)

In [3]:
def distortText(text, swap_type = "symboldelete"):
       
    # Distortion types
    distortions_symbols = ["symbolswap","symboldelete","symbolreplacerandom","symbolreplaceprev"]
    distortions_words = ["wordtrimright", "wordswapprev"]
    distortions_sentences = ["deletespaces"]
    # Symbol operations: max 2 symbols per word, max 2 symbol distortion types per word
    # Word trim: max 1 per word
    # Delete spaces: max all spaces, min 0 spaces
    
    list_words = text.split()
    
    distortion_prob_symbol = 0.5
    for i in range(len(list_words)):
        r = random.random()
        if swap_type == "symbolswap":
            # SYMBOL SWAP
            ## min distorted symbol = 0, max distorted symbol = len-2 because of indexing with 0 and this is the 1st swappable symbol
            distorted_symbol = round((len(list_words[i]) - 2) * random.random())
            s1 = list_words[i][distorted_symbol + 1]
            #print("{}---{}-{}-{}-{}".format(distorted_symbol, list_words[i][0:distorted_symbol - 1], list_words[i][distorted_symbol], s1, list_words[i][distorted_symbol + 1:]))
            #print(list_words[i][0:(distorted_symbol - 1) * (distorted_symbol > 0)])
            list_words[i] = list_words[i][0:distorted_symbol] + s1 + list_words[i][distorted_symbol] + list_words[i][distorted_symbol + 2:]
        elif swap_type == "symboldelete":
            # SYMBOL DELETE
            ## min distorted symbol = 0, max distorted symbol = len-1
            distorted_symbol = round((len(list_words[i]) - 1) * random.random())
            list_words[i] = list_words[i][0:distorted_symbol] + list_words[i][distorted_symbol + 1:]
        elif swap_type == "symbolreplacerandom":
            symbols = "abcdefghijklmnopqrstuvwxyz"
            # SYMBOL REPLACE RANDOM
            ## min distorted symbol = 0, max distorted symbol = len-1
            distorted_symbol = round((len(list_words[i]) - 1) * random.random())
            replace = symbols[round((len(symbols) - 1) * random.random())]
            list_words[i] = list_words[i][0:distorted_symbol] + replace + list_words[i][distorted_symbol + 1:]
        elif swap_type == "symbolreplaceprev":
            # SYMBOL REPLACE PREVIOUS
            ## min distorted symbol = 1, max distorted symbol = len - 1
            distorted_symbol = round(1 + (len(list_words[i]) - 2) * random.random())
            #print("{} - {}".format(distorted_symbol, list_words[i]))
            #print("{}---{}-{}-{}".format(distorted_symbol, list_words[i][0:(distorted_symbol)], list_words[i][distorted_symbol - 1], list_words[i][distorted_symbol + 1 * (len(list_words[i]) > distorted_symbol):distorted_symbol + 1 * (len(list_words[i]) > distorted_symbol)]))
            list_words[i] = list_words[i][0:(distorted_symbol)] + list_words[i][distorted_symbol - 1] + list_words[i][distorted_symbol + 1:]
        
        if swap_type == "wordtrimright":
            # WORD TRIM RIGHT
            ## min symbol = 1, max distorted symbol = len - 1
            trim_from = round(1 + (len(list_words[i]) - 1) * random.random())
            list_words[i] = list_words[i][:trim_from]
    
    if swap_type == "wordswapprev":
        # WORD SWAP
        word_id = round((len(list_words) - 2) * random.random())
        s1 = list_words[word_id]
        list_words[word_id] = list_words[word_id + 1]
        list_words[word_id + 1] = s1
        
    if swap_type == "deletespaces":
        # DELETE WHITESPACES
        ## -2 because there are len - 1 spaces in total
        space_id = round((len(list_words) - 2) * random.random())
        list_words[space_id] = list_words[space_id] + list_words[space_id + 1]
        list_words.pop(space_id + 1)
            
    distortedText = " ".join(list_words)
    return distortedText

In [4]:
text = "Antons likes pizza"
print(distortText(text=text,swap_type="symbolswap"))

print(distortText(text=text,swap_type="symboldelete"))

print(distortText(text=text,swap_type="symbolreplacerandom"))

print(distortText(text=text,swap_type="symbolreplaceprev"))

print(distortText(text=text,swap_type="wordtrimright"))

print(distortText(text=text,swap_type="wordswapprev"))

print(distortText(text=text,swap_type="deletespaces"))

Atnons lieks pziza
ntons like pzza
bntons lwkes puzza
Antoos likee ppzza
Anto like pi
Antons pizza likes
Antons likespizza


In [5]:
wikipedia = Wikipedia(
    language="simple",
    cache_directory_url=False
)

simplewiki-latest-pages-articles-multistream.xml.bz2
Loading parsed documents.
Loading preprocessed documents.
Wikipedia loaded.


In [6]:
# Cleaning up simple wikipedia texts
pattern_ignored_words = re.compile(
    r"""
    (?:(?:thumb|thumbnail|left|right|\d+px|upright(?:=[0-9\.]+)?)\|)+
    |^\s*\|.+$
    |^REDIRECT\b""",
    flags=re.DOTALL | re.UNICODE | re.VERBOSE | re.MULTILINE)
pattern_new_lines = re.compile('[\n\r ]+', re.UNICODE)
texts = [wikipedia.documents[i].text for i in range(len(wikipedia.documents))]
texts = [pattern_ignored_words.sub('', texts[i]) for i in range(len(texts))]
texts = [pattern_new_lines.sub(' ', texts[i]) for i in range(len(texts))]
texts = [texts[i].replace("\\", "") for i in range(len(texts))]
texts = [texts[i].replace("\xa0", " ") for i in range(len(texts))]

In [7]:
# Simple wikipedia article texts into single sentences

sentences = []
sentences += [tokenize.sent_tokenize(texts[i]) for i in range(len(texts))]
#sentences += [texts[i].split(". ") for i in range(len(texts))] #len(texts)
# Now sentences is a list of lists. The next expression flattens it into one long list.
sentences = [item for sublist in sentences for item in sublist]

In [8]:
pprint.pprint(sentences[0:3])

['The Month Spring flowers in April in the Northern Hemisphere.',
 'April comes between March and May, making it the fourth month of the year.',
 'It also comes first in the year out of the four months that have 30 days, as '
 'June, September and November are later in the year.']


In [9]:
print(len(sentences))
for i in reversed(range(len(sentences))):
    if len(sentences[i]) < 20 or len(sentences[i]) > 100 \
        or sentences[i][0:9] == "Category:" \
        or sentences[i][0:13] == "Related pages" \
        or sentences[i][0:10] == "References" \
        or sentences[i][0:14] == "Other websites":
        sentences.pop(i)
print(len(sentences))

#Gallery

1142223
648280


In [10]:
pprint.pprint(sentences[530000:530005])

['McCarty later explained the work for the general reader.',
 'Uses Propene is produced from fossil fuels, and from coal.',
 'Propene is the second most important product used in the petrochemical '
 'industry, after Ethene.',
 'About two thirds are used to produce Polypropylene.',
 'Propene and benzene are converted to acetone and phenol via the cumene '
 'process.']


In [11]:
import statistics
sentence_lengths = [len(sentences[i]) for i in range(len(sentences))]
print(statistics.median(sentence_lengths))

from collections import defaultdict
appearances = defaultdict(int)

sentence_lengths.sort()

for curr in sentence_lengths:
    appearances[curr] += 1
    
a = set(sentence_lengths) 
for i in a:
    print("{} - {}".format(i, appearances[i]))

63.0
20 - 2262
21 - 2398
22 - 6644
23 - 2670
24 - 3251
25 - 3208
26 - 3194
27 - 3488
28 - 3590
29 - 3899
30 - 4177
31 - 4754
32 - 4765
33 - 5145
34 - 5282
35 - 5572
36 - 5968
37 - 6002
38 - 6286
39 - 6727
40 - 6990
41 - 7439
42 - 7753
43 - 8142
44 - 8835
45 - 9162
46 - 9368
47 - 9875
48 - 10349
49 - 10546
50 - 10651
51 - 10699
52 - 10742
53 - 10845
54 - 10767
55 - 10812
56 - 10723
57 - 10611
58 - 10619
59 - 10610
60 - 10586
61 - 10580
62 - 10482
63 - 10394
64 - 10473
65 - 10232
66 - 10094
67 - 10029
68 - 10016
69 - 9925
70 - 9713
71 - 9810
72 - 9634
73 - 9984
74 - 9663
75 - 9281
76 - 9452
77 - 9147
78 - 9420
79 - 8992
80 - 8905
81 - 8782
82 - 8805
83 - 8615
84 - 8441
85 - 8453
86 - 8247
87 - 8388
88 - 8352
89 - 7819
90 - 7911
91 - 7842
92 - 7697
93 - 7515
94 - 7169
95 - 7359
96 - 7289
97 - 7207
98 - 7034
99 - 6955
100 - 6768


In [12]:
alphabets = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7, 'i':8, 'j':9, 'k':10, 'l':11, 'm':12, 'n':13, 'o':14,
            'p':15, 'q':16, 'r':17, 's':18, 't':19, 'u':20, 'v':21, 'w':22, 'x':23, 'y':24, 'z':25, 
            '0':26, '1':27, '2':28, '3':29, '4':30, '5':31, '6':32, '7':33, '8':34, '9':35, 
            ' ':36, ',':37, '.':38, ':':39, ';':40, '"':41, "'":42, '':43, '(':44, ')':45} #43 = unknown symbol

idxs = [alphabets[ch] if ch in alphabets else 43 for ch in 'az 123#']

idxs

#one_hot = tf.one_hot(idxs, depth=len(alphabets), dtype=tf.uint8)

#sess = tf.InteractiveSession()
#one_hot.eval()
one_hot = to_categorical(idxs, num_classes = len(alphabets))
one_hot

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 

In [13]:
sentences_idxs = []
for i in range(len(sentences)):
    idx = []
    for j in sentences[i]:
        if j in alphabets:
            idx += [alphabets[j]]
        else:
            idx += [43]
    sentences_idxs.append(idx)
    
#sentences_onehot = [tf.one_hot(sentences_idxs[i], depth=len(alphabets), dtype=tf.uint8) for i in range(len(sentences_idxs))]

In [14]:
#sentences_onehot = [tf.one_hot(sentences_idxs[i], depth=len(alphabets), dtype=tf.uint8) for i in range(10000)]
sentences_onehot = [to_categorical(sentences_idxs[i], num_classes = len(alphabets)) for i in range(10000)]
sentences = sentences[0:10000]

In [15]:
# Generate the data examples
# X and Y are identical for the test purposes

data = pd.DataFrame(
    {'X': sentences_onehot,
     'Y': sentences
    })

print(len(sentences_onehot[100][0]))
print(len(sentences_onehot))

46
10000


In [16]:
len(sentences_idxs)

648280

In [17]:
#####################################################

import os

import keras
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, Nadam

#from KerasDeepSpeech.data import combine_all_wavs_and_trans_from_csvs
from KerasDeepSpeech.generator import BatchGenerator
from KerasDeepSpeech.model import *
from KerasDeepSpeech.report import ReportCallback
from KerasDeepSpeech.utils import load_model_checkpoint, save_model, MemoryCallback

#####################################################


#######################################################

# Prevent pool_allocator message
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#######################################################


def main(args):
    '''
    There are 5 simple steps to this program
    '''

    #1. combine all data into 2 dataframes (train, valid)
    print("Getting data from arguments")
    #train_dataprops, df_train = combine_all_wavs_and_trans_from_csvs(args.train_files, sortagrad=args.sortagrad)
    #valid_dataprops, df_valid = combine_all_wavs_and_trans_from_csvs(args.valid_files, sortagrad=args.sortagrad)

    train_ratio = 0.9
    args.model_arch = 0
    args.opt = "adam"
    args.train_steps = 0
    args.epochs = 10
    args.valid_steps = 0
    args.batchsize = 32 #was 16
    args.name = ""
    args.loadcheckpointpath = ""
    args.fc_size = 512
    args.rnn_size = 512
    args.learning_rate = 0.01
    args.memcheck = False
    args.tensorboard = True
    
    model_input_type = "text"
    
    
    df_train = data[0:int(train_ratio * len(sentences_onehot))]
    df_valid = data[int(train_ratio * len(sentences_onehot)) + 1:]


    ## 2. init data generators
    print("Creating data batch generators")
    traindata = BatchGenerator(dataframe=df_train, dataproperties=None,
                              training=True, batch_size=args.batchsize, model_input_type=model_input_type)
    validdata = BatchGenerator(dataframe=df_valid, dataproperties=None,
                              training=False, batch_size=args.batchsize, model_input_type=model_input_type)




    output_dir = os.path.join('checkpoints/results',
                                  'model%s_%s' % (args.model_arch,
                                             args.name))
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)


    ## 3. Load existing or create new model
    if args.loadcheckpointpath:
        # load existing
        print("Loading model")

        cp = args.loadcheckpointpath
        assert(os.path.isdir(cp))

        model_path = os.path.join(cp, "model")
        # assert(os.path.isfile(model_path))

        model = load_model_checkpoint(model_path)


        print("Model loaded")
    else:
        # new model recipes here
        print('New model DS{}'.format(args.model_arch))
        if (args.model_arch == 0):
            # DeepSpeech1 with Dropout
            model = ds1_dropout(input_dim=len(alphabets), fc_size=args.fc_size, rnn_size=args.rnn_size,dropout=[0.1,0.1,0.1], output_dim=len(alphabets) + 1)

        elif(args.model_arch==1):
            # DeepSpeech1 - no dropout
            model = ds1(input_dim=26, fc_size=args.fc_size, rnn_size=args.rnn_size, output_dim=29)

        elif(args.model_arch==2):
            # DeepSpeech2 model
            model = ds2_gru_model(input_dim=161, fc_size=args.fc_size, rnn_size=args.rnn_size, output_dim=29)

        elif(args.model_arch==3):
            # own model
            model = ownModel(input_dim=26, fc_size=args.fc_size, rnn_size=args.rnn_size, dropout=[0.1, 0.1, 0.1], output_dim=29)

        elif(args.model_arch==4):
            # graves model
            model = graves(input_dim=26, rnn_size=args.rnn_size, output_dim=29, std=0.5)

        elif(args.model_arch==5):
            #cnn city
            model = cnn_city(input_dim=161, fc_size=args.fc_size, rnn_size=args.rnn_size, output_dim=29)

        elif(args.model_arch == 6):
            # constrained model
            model = const(input_dim=26, fc_size=args.fc_size, rnn_size=args.rnn_size, output_dim=29)
        else:
            raise("model not found")

        print(model.summary(line_length=80))

        #required to save the JSON
        save_model(model, output_dir)

    if (args.opt.lower() == 'sgd'):
        opt = SGD(lr=args.learning_rate, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
    elif (args.opt.lower() == 'adam'):
        opt = Adam(lr=args.learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-8, clipnorm=5)
    elif (args.opt.lower() == 'nadam'):
        opt = Nadam(lr=args.learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-8, clipnorm=5)
    else:
        raise "optimiser not recognised"

    model.compile(optimizer=opt, loss=ctc)

    ## 4. train

    if args.train_steps == 0:
        args.train_steps = len(df_train.index) // args.batchsize
        # print(args.train_steps)
    # we use 1/xth of the validation data at each epoch end to test val score
    if args.valid_steps == 0:

        args.valid_steps = (len(df_valid.index) // args.batchsize)
        # print(args.valid_steps)


    if args.memcheck:
        cb_list = [MemoryCallback()]
    else:
        cb_list = []

    if args.tensorboard:
        tb_cb = TensorBoard(log_dir='./tensorboard/{}/'.format(args.name), write_graph=False, write_images=True)
        cb_list.append(tb_cb)

    y_pred = model.get_layer('ctc').input[0]
    input_data = model.get_layer('the_input').input

    report = K.function([input_data, K.learning_phase()], [y_pred])
    report_cb = ReportCallback(report, validdata, model, args.name, save=True)

    cb_list.append(report_cb)

    model.fit_generator(generator=traindata.next_batch(),
                        steps_per_epoch=args.train_steps,
                        epochs=args.epochs,
                        callbacks=cb_list,
                        validation_data=validdata.next_batch(),
                        validation_steps=args.valid_steps,
                        initial_epoch=0,
                        verbose=1,
                        class_weight=None,
                        max_q_size=10,
                        workers=1,
                        pickle_safe=False
                        )

    # K.clear_session()

    ## These are the most important metrics
    print("Mean WER   :", report_cb.mean_wer_log)
    print("Mean LER   :", report_cb.mean_ler_log)
    print("NormMeanLER:", report_cb.norm_mean_ler_log)

    # export to csv?
    K.clear_session()

In [18]:
class Object(object):
    pass

args = Object()

In [ ]:
main(args)

Getting data from arguments
Creating data batch generators
New model DS0
________________________________________________________________________________
Layer (type)              Output Shape      Param #  Connected to               
the_input (InputLayer)    (None, None, 46)  0                                   
________________________________________________________________________________
time_distributed_1 (TimeD (None, None, 512) 24064    the_input[0][0]            
________________________________________________________________________________
time_distributed_2 (TimeD (None, None, 512) 0        time_distributed_1[0][0]   
________________________________________________________________________________
time_distributed_3 (TimeD (None, None, 512) 262656   time_distributed_2[0][0]   
________________________________________________________________________________
time_distributed_4 (TimeD (None, None, 512) 0        time_distributed_3[0][0]   
____________________________________

247/281 [=========================>....] - ETA: 10:38 - loss: in - ETA: 6:03 - loss: in - ETA: 4:31 - loss: i - ETA: 3:45 - loss: i - ETA: 3:17 - loss: i - ETA: 2:58 - loss: i - ETA: 2:44 - loss: i - ETA: 2:35 - loss: i - ETA: 2:27 - loss: i - ETA: 2:21 - loss: i - ETA: 2:15 - loss: i - ETA: 2:11 - loss: i - ETA: 2:07 - loss: i - ETA: 2:04 - loss: i - ETA: 2:01 - loss: i - ETA: 1:59 - loss: i - ETA: 1:56 - loss: i - ETA: 1:54 - loss: i - ETA: 1:52 - loss: i - ETA: 1:51 - loss: i - ETA: 1:49 - loss: i - ETA: 1:48 - loss: i - ETA: 1:46 - loss: i - ETA: 1:45 - loss: i - ETA: 1:44 - loss: i - ETA: 1:43 - loss: i - ETA: 1:42 - loss: i - ETA: 1:41 - loss: i - ETA: 1:40 - loss: i - ETA: 1:39 - loss: i - ETA: 1:38 - loss: i - ETA: 1:37 - loss: i - ETA: 1:36 - loss: i - ETA: 1:35 - loss: i - ETA: 1:35 - loss: i - ETA: 1:34 - loss: i - ETA: 1:33 - loss: i - ETA: 1:32 - loss: i - ETA: 1:32 - loss: i - ETA: 1:31 - loss: i - ETA: 1:30 - loss: i - ETA: 1:30 - loss: i - ETA: 1:29 - loss: i - ETA: 1:2

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())